# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

In [5]:
import sklearn # Проверка версии sklearn. Обновляем при необходимости
if sklearn.__version__[0] == '0':
    !pip install scikit-learn -U # После установки надо перезапустить ядро
sklearn.__version__ 

'1.5.1'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import notebook
try:
    import torch
except:
    !pip install torch
    import torch

try:
    import transformers
except:
    !pip install transformers
    import transformers

try:
    from ydata_profiling import ProfileReport
except:
    !pip install -U Pillow # Для решения ошибки с профайлером
    !pip install -U ydata-profiling
    from ydata_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
RANDOM_STATE = 42

## Подготовка

Для начала загрузим данные и сразу-же с ними ознакомимся.

In [8]:
try:
    ds = pd.read_csv('/datasets/toxic_comments.csv', index_col='Unnamed: 0')
except:
    ds = pd.read_csv('datasets/toxic_comments.csv', index_col='Unnamed: 0')
ds = ds.sample(400, random_state=RANDOM_STATE).reset_index(drop=True)
ProfileReport(ds)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Дизбаланс классов в целевом признаке, токсичных комментариев почти в 10 раз меньше, чем нетоксичных. Без стратификации здесь не обойтись. 

Так-же не обойтись и без фильтра стоп-слов, они встречаются в наших текстах чаще всех остальных.

Поделим данные на выборки. Тестовую отложим для лучшей модели, а под тренировочную начнём подгонять трансформеры признаков.

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')  
model = transformers.BertModel.from_pretrained('bert-base-cased')

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [10]:
class ToxicFeatures(TransformerMixin, BaseEstimator):
    def __init__(self, max_len=512):
        self.max_len = max_len
    
    def fit(self, X, y): 
        # Обучение классу не требуется, скипаем
        return self
    
    def transform(self, X): 
        # Токенизация
        tokenized = X.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=self.max_len, truncation=True))
        max_len = max(tokenized.apply(len))
        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
        attention_mask = np.where(padded != 0, 1, 0)
        batch_size = 100
        embeddings = []
        for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
            attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
            
            with torch.no_grad():
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)
            
            embeddings.append(batch_embeddings[0][:,0,:].numpy())
        return np.concatenate(embeddings)

In [11]:
tf = ToxicFeatures()

X_train, X_test, y_train, y_test = train_test_split(
    tf.transform(ds['text']), 
    ds['toxic'], 
    stratify=ds['toxic'], 
    test_size=0.2, 
    random_state=RANDOM_STATE)

  0%|          | 0/4 [00:00<?, ?it/s]

## Обучение

In [12]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [13]:
f1_score(y_test, logreg.predict(X_test))

0.5714285714285714

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны